# Assigment 1
### 02467 Computational Social Science Group 6

## Part 1: Web-scraping
### _Exercise: Web-scraping the list of participants to the International Conference in Computational Social Science_

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

def clean_name(name):
    # Clean name - remove special characters and unnecessary whitespace
    name = re.sub(r'[,\n\t\r]', '', name)
    name = re.sub(r'\s+', ' ', name)
    name = name.strip()
    name = re.sub(r'</?u>', '', name)

    # Exclude names that are too short or contain digits
    if len(name) < 2 or bool(re.search(r'\d', name)):
        return None

    return name

def extract_names_from_text(text):
    # Extract names from text
    name_list = []
    if ',' in text:
        author_list = text.split(',')
        for author in author_list:
            name = clean_name(author)
            if name:
                name_list.append(name)
    else:
        name = clean_name(text)
        if name:
            name_list.append(name)
    return name_list

def parse_html_for_names(html_content):
    # Extract all researcher names from HTML
    soup = BeautifulSoup(html_content, 'html.parser')
    name_list_to_set = set()

    # Find all list items containing presentation titles and author information
    for li in soup.find_all('li'):
        text = li.get_text()
        if not text:
            continue

        # Find <i> tags containing author information
        authors_tag = li.find('i')
        if authors_tag:
            authors_text = authors_tag.get_text()
            name_list = extract_names_from_text(authors_text)
            name_list_to_set.update(name_list)

    # Find session chairs
    chair_patterns = soup.find_all(string=re.compile(r'Chair:', re.IGNORECASE))
    for pattern in chair_patterns:
        chair_text = pattern.strip()
        if 'Chair:' in chair_text:
            chair_name = chair_text.split('Chair:')[1]
            name = clean_name(chair_name)
            if name:
                name_list_to_set.add(name)

    return sorted(list(name_list_to_set))

def main():
    # URL of the conference program
    url = "https://2023.ic2s2.org/program"

    # Fetch HTML content from the URL
    response = requests.get(url)

    if response.status_code == 200:
        html_content = response.text
        # Extract names
        names = parse_html_for_names(html_content)

        # Save results to a file
        output_path = 'ic2s2_2023_researchers.txt'
        with open(output_path, 'w', encoding='utf-8') as f:
            for name in names:
                f.write(name + '\n')

        print(f"A total of {len(names)} unique researcher names have been extracted.")
        print(f"Results have been saved to {output_path}.")
    else:
        print(f"Failed to fetch the webpage. Status code: {response.status_code}")

if __name__ == "__main__":
    main()

### _How many unique researchers do you get?_
#### We got 1484 unique researchers for our answer.

### _Explain the process you followed to web-scrape the page. Which choices did you make to accurately retreive as many names as possible? Which strategies did you use to assess the quality of your final list? Explain your reasoning and your choices (answer in max 150 words)._


To web-scrape the conference webpage, I used the `requests` library to fetch the HTML content and `BeautifulSoup` to parse it. I inspected the webpage structure to identify relevant elements containing researcher names, such as `` tags for authors and `` tags for author details. I also searched for text patterns like "Chair:" to extract session chairs. Names were cleaned using regular expressions to remove unwanted characters, whitespace, and duplicates.

To maximize accuracy, I handled different name formats (e.g., comma-separated lists) and ensured all names included both first and last names. I used a set to store names, avoiding duplicates caused by slight variations in formatting.

To assess quality, I manually reviewed a sample of extracted names for completeness and correctness. The cleaning process ensured no invalid or partial names were included, resulting in a comprehensive and accurate list of unique contributors.

## Part 2: Ready Made vs Custom Made Data

### Centola
#### Pros:
#### - Flexibility, data collection in real time can be modified
#### - Custom made data can be designed to address specific questions
#### Cons:
#### - Social network is something complex, researchers might accidentally introduce bias
#### - Might not fully represent real world conditions


### Nicolaide
#### Pros:
#### - Saves time and cost
#### - Larger sample size
#### Cons:
#### - Quality of data might not be the best
#### - Researchers do not have control over how the data was made

### _How do you think these differences can influence the interpretation of the results in each study?_
#### Centola allows for controlled experiments with specific manipulations while Nicolaides provide a real world view. However, Centola’s study might not be generalisable to the general population, while Nicolaide’s study may have causal correlations and uncertainty.

## Part 3: Gathering Research Articles using the OpenAlex API

In [ ]:
import requests
import pandas as pd
from joblib import Parallel, delayed

df = pd.read_csv('file02.csv')
base_url = "https://api.openalex.org/"
papers = []
abstracts = []

socialscience = {
    "Sociology": "https://openalex.org/C144024400",
    "Psychology": "https://openalex.org/C15744967",
    "Economics": "https://openalex.org/C162324750",
    "Political Science": "https://openalex.org/C17744445"
}

quantitative = {
    "Mathematics": "https://openalex.org/C33923547",
    "Physics": "https://openalex.org/C121332964",
    "Computer Science": "https://openalex.org/C41008148"
}

def getwork(works_api_url, per_page=200):
    works = []
    cursor = "*"  # initial cursor
    while True:
        url = f"{works_api_url}&per-page={per_page}&cursor={cursor}"
        response = requests.get(url)
        if response.status_code != 200: break

        data = response.json()
        works.extend(data.get('results', []))

        # next cursor for pagination
        next_cursor = data.get('meta', {}).get('next_cursor')
        if not next_cursor: break
        cursor = next_cursor
    return works

def filtering(works):
    filtered = []
    for work in works:
        if work.get('cited_by_count', 0) <= 10: continue  #more than 10 citations
        if len(work.get('authorships', [])) >= 10: continue #fewer than 10 authors

        concept_ids = [concept.get('id') for concept in work.get('concepts', [])]
        is_SS = any(concept in socialscience.values() for concept in concept_ids)
        is_quant = any(concept in quantitative.values() for concept in concept_ids)

        if is_SS and is_quant: #works relevant to Computational Social Science  AND intersecting with a quantitative discipline
            filtered.append(work)
    return filtered

def filtering2(id, worksurl, count): # Only if the author has between 5 and 5000 works
    if 5 <= count <= 5000:
        works = getwork(worksurl)
        filtered_works = filtering(works)
        return filtered_works
    return []

def extraction(work):
    return {
        'id': work.get('id'),
        'publication_year': work.get('publication_year'),
        'cited_by_count': work.get('cited_by_count'),
        'author_ids': [author.get('author', {}).get('id') for author in work.get('authorships', [])],
        'title': work.get('title'),
        'abstract_inverted_index': work.get('abstract_inverted_index'),
        'referenced_works': work.get('referenced_works', []),
        'cited_by_api_url': work.get('cited_by_api_url'),
        'related_works': work.get('related_works', [])
    }

# parallelize fetching and filtering works using joblib
allfiltered = Parallel(n_jobs=-1)(
    delayed(filtering2)(row['OpenAlex ID'], row['Works API URL'], row['Works Count'])
    for _, row in df.iterrows()
)

for all in allfiltered:
    for work in all:
        details = extraction(work)

        papers.append({
            'id': details['id'],
            'publication_year': details['publication_year'],
            'cited_by_count': details['cited_by_count'],
            'author_ids': details['author_ids'],
            'referenced_works': details['referenced_works'],
            'cited_by_api_url': details['cited_by_api_url'],
            'related_works': details['related_works']
        })

        abstracts.append({
            'id': details['id'],
            'title': details['title'],
            'abstract_inverted_index': details['abstract_inverted_index']
        })

papers_df = pd.DataFrame(papers)
abstracts_df = pd.DataFrame(abstracts)

papers_df.to_csv('papers.csv', index=False)
abstracts_df.to_csv('abstracts.csv', index=False)
print("Data saved.")

### Data Overview and Reflection questions:

### _Dataset summary. How many works are listed in your IC2S2 papers dataframe? How many unique researchers have co-authored these works?_
#### Number of works: 11230
#### Number of unique researchers: 15199

### _Efficiency in code. Describe the strategies you implemented to make your code more efficient. How did your approach affect your code's execution time?_
#### As suggested, I used joblib's parallel function to handle multiple requests. I also implemented the filters required (work count between 5-5000, works with more than 10 citations, works authored by fewer than 10 individuals and works related to computational social science). This allowed the code to be executed at a faster rate.

### _Filtering Criteria and Dataset Relevance Reflect on the rationale behind setting specific thresholds. How do these filtering criteria contribute to the relevance of the dataset you compiled?_
#### Work count filter: >5 work count allows us to focus on established authors, while <5000 work count removes authors who have too many work that could otherwise produce unwanted "noise"
#### More than 10 citations: citations allow us to judge the influence of a work, and this filter will allow us to get datasets that has some sort of influence in the academic field
#### Less than 10 authors per work: <10 authors would suggest that the work was collaborative and focused, as too many authors may cause a paper to have too many ideas and may not reflect clear insights
#### Relevance to Computational Social Science: as the course is related to computational social science, it is important that the works we find are related to it

### _Do you believe any aspects of Computational Social Science research might be underrepresented or overrepresented as a result of these choices?_
#### Yes, I believe that there may be some sort of underrepresentation due to such filters.
#### Firstly, there may be newly published work that currently have <10 citations but is still highly relevant to the field. The filter would then exclude such works which may sometimes be highly niche that results in the lower citation count. Secondly, filtering of <10 authors could potentially exclude works that were done by a large team. It could have been a complex topic that required a large team to collaborate on, which would still be relevant to the field but are excluded due to such filters.